### Gridflow process
This walkthrough highlights the main steps: importing dependencies, configuring inputs, partitioning countries into zones, summarizing zonal data, building the flow network, and producing EPM-ready CSVs.

#### Step 1: imports (see `README.md` for installation hints)
Only bring in the plotting tools and Gridflow modules that the notebook actually uses; consult `README.md` if you encounter dependency issues.

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

from pathlib import Path

from gridflow import model
from gridflow import epm_input_generator
from gridflow.visuals import country_viz

plt.style.use('fivethirtyeight')

#### Step 2: user inputs
Centralize the dataset paths, countries, and modeling configuration so you can adjust everything before running the workflow.

In [ ]:
# dataset locations and region configuration
data_path = 'data/global_datasets'  # path containing borders, rasters, and grid.gpkg

countries = ["LUX"]  # ISO3 codes for the countries to model
n_zones = 5  # target number of zones per the region

# epm
epm_input_raw = 'data/epm_inputs_raw'  # raw EPM CSV templates
epm_output_dir = 'data/epm_inputs'  # destination for zonalized CSVs

#### Step 3: build the region and create zones
Instantiate the region object for the selected countries and segment it into the requested number of zones to prepare the spatial structure.

In [ ]:
region = model.region(countries, data_path)
region.create_zones(n=n_zones)

#### Step 4: inspect the zoning results
Review the geopandas table that describes each zone so you can verify the segmentation before enriching it.

In [ ]:
region.zones

#### Step 5: enrich zones and compute statistics
Populate the zones with aggregated datasets (population, renewables) and use the built-in stats to summarize key metrics.

In [ ]:
region.set_zone_data()
region.zone_stats

#### Step 6: create and visualize the network
Generate the zone-to-zone flow network, visualize it on the country map, and display the computed flow matrix.

In [ ]:
region.create_network()
country_viz(region)

plt.figure(figsize=(6, 4))
plt.imshow(region.grid.flow)
plt.title('Flow field for the generated network')
plt.colorbar()

#### Step 7: auto-transform inputs for EPM
Once the zone split and network are defined, run the EPM input generator to reshape the raw CSV templates into zonal files that match `region.zones`.

In [ ]:
epm_input_generator.generate_epm_inputs(region, epm_input_raw, epm_output_dir)
print(f'Zonal inputs written to {Path(epm_output_dir).resolve()}')

#### Next steps
Adjust the user inputs, explore additional metrics, or export region-aware artifacts for downstream modeling after these core steps complete.